<a href="https://colab.research.google.com/github/pmontman/tmp_choicemodels/blob/main/nb/tutorials/WK_07_tuto_panel_mixed_logit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial 7: Panel data and mixed logit

We will analyze a marketing dataset, of choice of brand of catsup (a.k.a. ketchup or 'tomato sauce' in Australia).

We have two famous brands of catsup and 3 different package sizes.

A description of the dataset can be found [here](https://www.tandfonline.com/doi/pdf/10.1080/07350015.1994.10524547?casa_token=r4LpjVvgDW4AAAAA:FVG8mEexsQ37tJ2bvk7oxZZ9K_jvvMJ2WxglLzBaHQD0_0REkXmKGsPPxXw_LRGwN3YHY8-L-k8U)

# Description of the dataset

* **id**: household identifiers,
* **choice**: one of heinz41, heinz32, heinz28, hunts32.
* **disp_x**: is there a display for brand X ?
* **feat_x**: is there a newspaper feature advertisement for brand x?
* **price_x**: price of brand x

---
---

# Preparing the environment
*The preparation and dataset loading code is given to the students*

In [1]:
!pip install biogeme

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 5.1 MB/s 
  Created wheel for biogeme: filename=biogeme-3.2.10-cp37-cp37m-linux_x86_64.whl size=4253426 sha256=2b5340f022e91416de63e01f1eb71c0da0e9e3c556cf6502489097a681615e60
  Stored in directory: /root/.cache/pip/wheels/5b/92/9b/63caa7ad9b2cd582de77d3701d10f7e8d041466f4a9d07d554
Successfully built biogeme


Load the packages, feel free to change the names.

In [2]:
import pandas  as pd
import numpy as np
import matplotlib.pyplot as plt

import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import biogeme.expressions as exp
import biogeme.tools as tools
import biogeme.distributions as dist

# Load the dataset

In [3]:
path = 'https://raw.githubusercontent.com/pmontman/pub-choicemodels/main/data/catsup.csv'
catsup_pd = pd.read_csv(path)


In this case, notice the id variable, that identifies each household, so we have data from many choice situations for each household. We have also different amount of observations per household.

In [4]:
catsup_pd.head(25)

,id,disp_heinz41,disp_heinz32,disp_heinz28,disp_hunts32,feat_heinz41,feat_heinz32,feat_heinz28,feat_hunts32,price_heinz41,price_heinz32,price_heinz28,price_hunts32,choice
0,1,0,0,0,0,0,0,0,0,4.6,3.7,5.2,3.4,heinz28
1,1,0,0,0,0,0,0,0,0,4.6,4.3,5.2,4.4,heinz28
2,1,0,0,0,0,0,1,0,0,4.6,2.5,4.6,4.8,heinz28
3,1,0,0,0,0,0,0,0,0,4.6,3.7,5.2,3.4,heinz28
4,1,0,0,0,0,0,0,1,0,4.6,3.0,4.6,4.8,heinz28
5,1,0,0,0,0,0,0,0,0,5.0,3.0,4.7,3.0,heinz28
6,1,0,0,0,1,0,0,0,1,5.1,3.1,4.6,4.1,heinz28
7,1,0,0,0,0,0,0,0,0,4.6,3.4,4.7,3.1,heinz41
8,1,0,0,0,0,0,0,0,0,5.0,3.4,4.7,3.1,heinz28
9,1,0,0,0,1,0,0,0,0,5.0,3.4,5.0,2.8,heinz28


# Auxiliary function

In [5]:
def qbus_update_globals_bgm(pd_df):
   globals().update(db.Database('tmp_bg_bgm_for_glob', pd_df).variables)

# Data cleaning: Preparing the dataset for Biogeme

Encode the choice variable (a string) into numbers with the `factorize` function.
We take a look at the codetable to know how the numbers are mapped to the alternatives. The order of the codetable indicates the association,
0: heinz28, 1: heinz41 and so on.

In [6]:
catsup_pd['choice'], codetable = catsup_pd['choice'].factorize()

In [7]:
codetable

Index(['heinz28', 'heinz41', 'heinz32', 'hunts32'], dtype='object')

#Capturing agent effect through mixed logit

In [8]:

# Define level of verbosity
import biogeme.messaging as msg
logger = msg.bioMessage()
logger.setSilent()


We will have to use some additional functionalities of biogeme so the auxiliary functions cannot be used. So we have to create the biogeme database manually.

In [9]:
database = db.Database("catsup", catsup_pd)

This line of code tells biogeme to use the variable id in the dataset as the identifier of the individuals to treat them as panel data (if not, it will take observations as if they were independent).

In [10]:
database.panel("id")

We now declare the coefficients in our model, we will make a simple model,
just the alternative-specif constants and the variables disp, feat and price.

In [11]:
ASC_heinz41 = exp.Beta('ASC_heinz41',0,None,None,0)
ASC_heinz32  = exp.Beta('ASC_heinz32 ',0,None,None,0)
ASC_heinz28 = exp.Beta('ASC_heinz28',0,None,None,1)
ASC_hunts32 = exp.Beta('ASC_hunts32',0,None,None,1)

B_disp = exp.Beta('B_disp',0,None,None,0)
B_feat = exp.Beta('B_feat',0,None,None,0)
B_price = exp.Beta('B_price',0,None,None,0)

Now comes the **important part**, the definition of the random effects in the model!

We want to consider a simple constant agent effect.
Recall the definition:

$$V_{jit} = \beta X_{jit} + \alpha_{ij}$$
and we have to specifiy the distribition of the $\alpha$. In this case, we will use the normal, so $\alpha_{ij} \in N(\mu_j, \sigma_j^2)$.
The ingredients are:
 1) Probability distrbitution: We set it ourselves, the normal (we could have set others, such as the uniform, lognormal and so on.
 2) Parameters of the probability distribution (the $\mu$ and $\sigma$): They will be estimated from the data.

 If we think for a moment, the $\mu$ of this distribution will also depend on the values of the alternative-specific constants (ASC), since changing $mu$ essential mean adding a constant to all values coming for that distribution. We can think that the mean of that distribution will be 'absorbed' by the ASCs.
Again this is just another convention!

What we end up doing in biogeme is that we will only declare the parameter for the standard deviation, so each normal distribution will be mean 0 and std.dev to determine from the data. 


In terms of code, we delcare the std.devs (the $\sigma_j$} just as any other parameter. We have to set on of them to 0 to act as reference (remember that changes in scale do not affect utility).

In [12]:

SIGMA_heinz41 = exp.Beta('SIGMA_heinz41',0,0,None,0)
SIGMA_heinz32 = exp.Beta('SIGMA_heinz32',0,0,None, 1)
SIGMA_heinz28 = exp.Beta('SIGMA_heinz28',0,0,None,0)
SIGMA_hunts32 = exp.Beta('SIGMA_hunts32',0,0,None,0)


And the following code is how we tell biogeme that the parameters are random.
The `EC_`s are the agent effects. The key function that indicate randomness is the biogeme function `exp.bioDraws`, that indicates that they are random, drawn from a probability distribution. The second argument specifies the distribution, some possible values are `'NORMAL'` `'UNIFORMSYM'`.

In [13]:

# Define random parameters, normally distributed across individuals,
# designed to be used for Monte-Carlo simulation
EC_heinz41 = SIGMA_heinz41 * exp.bioDraws('EC_heinz41','NORMAL')
EC_heinz32 = SIGMA_heinz32 * exp.bioDraws('EC_heinz32','NORMAL')
EC_heinz28 = SIGMA_heinz28 * exp.bioDraws('EC_heinz28','NORMAL')
EC_hunts32 = SIGMA_hunts32 * exp.bioDraws('EC_hunts32','NORMAL')

An this is how the specification of the utility functions looks like in the end.
The Betas (disp, feat and price) and ASCsare fixed effect, while the EC are random.

In [14]:
globals().update(database.variables)
# Definition of the utility functions
V_heinz41 = ASC_heinz41 + B_disp *disp_heinz41 + B_feat * feat_heinz41 + B_price * price_heinz41 + EC_heinz41
V_heinz32 = ASC_heinz32 + B_disp *disp_heinz32 + B_feat * feat_heinz32 + B_price * price_heinz32 + EC_heinz32
V_heinz28 = ASC_heinz28 + B_disp *disp_heinz28 + B_feat * feat_heinz28 + B_price * price_heinz28 + EC_heinz28
V_hunts32 = ASC_hunts32 + B_disp *disp_hunts32 + B_feat * feat_hunts32 + B_price * price_hunts32 + EC_hunts32

We create the dictionary that maps to the alternatives. **Remember to be careful here.** The numbers should match the alternatives as we indicated by the factorize() transformation at the beginning of the notebook.
Availabilities are not considered here, we set them to 1.
Finaly, we specify the logit model, as usual.
These steps are common to the multinomial logit.

In [15]:
# Associate utility functions with the numbering of alternatives
V = {0: V_heinz28,
     1: V_heinz41,
     2: V_heinz32,
     3: V_hunts32}

av = {0: 1,
     1: 1,
     2: 1,
     3: 1}

# Conditional to the random variables, the likelihood of one observation is
# given by the logit model (called the kernel)
obsprob = models.logit(V,av, choice)


The difference from the usual declaration of the MNL comes now.

We have to do two new steps:

1. Tell biogeme to consider the panel nature of the data.
2. Tell biogeme to calculate the choice probabilities by simulation. This is how we deal with the random parameters. We simulate for the distribution, and the we calculate the likelihood for that distribution.

Step 1: we can do it by modifying the model with the expression `exp.PanelLikelihoodTrajectory`.

In [16]:
condprobIndiv = exp.PanelLikelihoodTrajectory(obsprob)

And Step 2 we take the model and the modifyi it by the expresion `exp.MonteCarlo`. The final log it to take the loglikelihood.

In [17]:
logprob = exp.log(exp.MonteCarlo(condprobIndiv))

We we are using simulation, we have to tell biogeme how many draws from the distribution are we going to generate. The more draws, the more accurate estimation, but it is compuationally costly.

We also set up a seed, so we can get the same results if the run the notebook again (setting up a seed is a good habit in general)

In [18]:

# Create the Biogeme object
biogeme  = bio.BIOGEME(database,logprob,numberOfDraws=250, seed=1)




Estimation and results as usual.

In [19]:

# Estimate the parameters. 
results = biogeme.estimate()

We take a look at the results,

In [20]:
results.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_heinz32,1.083057,0.221825,4.882495,1.047517e-06
ASC_heinz41,0.681922,0.320186,2.129771,3.319053e-02
B_disp,1.256178,0.143741,8.739178,0.000000e+00
B_feat,1.265543,0.154216,8.206304,2.220446e-16
B_price,-1.699963,0.102527,-16.580691,0.000000e+00
SIGMA_heinz28,3.100125,0.249963,12.402331,0.000000e+00
SIGMA_heinz41,1.932301,0.188870,10.230847,0.000000e+00
SIGMA_hunts32,1.953489,0.288904,6.761723,1.363598e-11


Interestingly, the simulation of panel data is not implemented! No problem
we will still be able to do simulations, by setting up a scenario that does not consider the panel data.

In [21]:
#biogeme.simulate(results.getBetaValues())

# Compare Panel vs Not using the panel information

We will just compare the results that we get if the just ignore the panel information. We can recreate this by not using `exp.PanelLikelihoodTrajectory` in the model.

In math, this would be the specification:

$$V_{jit} = \beta X_{jit} + \alpha_{ij \color{red}{t}}$$

Os opposed to the panel specification
$$V_{jit} = \beta X_{jit} + \alpha_{ij}$$

Notice the difference subindex $t$.

In [22]:
database_nonpanel = db.Database("catsup", catsup_pd)


# We integrate over the random variables using Monte-Carlo
logprob_nonpanel = exp.log(exp.MonteCarlo(obsprob))


# Create the Biogeme object
biogeme_nonpanel  = bio.BIOGEME(database_nonpanel,logprob_nonpanel,numberOfDraws=250, seed=1)


# Estimate the parameters. 
results_nonpanel = biogeme_nonpanel.estimate()


In [23]:

results_nonpanel.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_heinz32,1.621548,0.092137,17.599401,0.000000e+00
ASC_heinz41,0.913188,0.376583,2.424931,1.531129e-02
B_disp,1.483782,0.215598,6.882153,5.895506e-12
B_feat,1.701913,0.263462,6.459813,1.048326e-10
B_price,-2.345118,0.157310,-14.907575,0.000000e+00
SIGMA_heinz28,6.645047,0.444695,14.942938,0.000000e+00
SIGMA_heinz41,2.490085,0.415468,5.993446,2.054410e-09
SIGMA_hunts32,0.110482,0.121289,0.910897,3.623498e-01


Simulation of the choice probabilities is similar to the MNL, we just have
to 

In [24]:
tgt_nonpanel = {
    0: exp.MonteCarlo(models.logit(V, av, 0)),
    1: exp.MonteCarlo(models.logit(V, av, 1)),
    2: exp.MonteCarlo(models.logit(V, av, 2)),
    3: exp.MonteCarlo(models.logit(V, av, 3))
    }


In [25]:

sim_nonpanel = bio.BIOGEME(database_nonpanel, tgt_nonpanel, numberOfDraws=250, seed=1)
preds = sim_nonpanel.simulate(theBetaValues=results_nonpanel.getBetaValues())
preds

,0,1,2,3
0,0.218093,0.107306,0.480553,0.194049
1,0.255563,0.233181,0.442584,0.068672
2,0.090402,0.008071,0.901379,0.000148
3,0.185884,0.109799,0.504567,0.199750
4,0.248235,0.059396,0.690354,0.002015
...,...,...,...,...
2793,0.211643,0.242937,0.388860,0.156559
2794,0.356240,0.059148,0.465895,0.118717
2795,0.152344,0.014330,0.066761,0.766565
2796,0.178328,0.045717,0.293072,0.482884


In [26]:
sim_panel = bio.BIOGEME(database_nonpanel, tgt_nonpanel, numberOfDraws=250, seed=1)
preds = sim_panel.simulate(theBetaValues=results.getBetaValues())
preds

,0,1,2,3
0,0.121112,0.130704,0.401088,0.347097
1,0.153719,0.249373,0.386964,0.209944
2,0.036706,0.022545,0.933244,0.007505
3,0.096594,0.134774,0.452346,0.316286
4,0.193218,0.096319,0.664599,0.045863
...,...,...,...,...
2793,0.115805,0.257673,0.332078,0.294444
2794,0.245640,0.076661,0.390784,0.286915
2795,0.075651,0.037966,0.166706,0.719677
2796,0.102782,0.069232,0.317206,0.510780


#Compare to the Multinomial Logit (fixed effects) without agent effect

Just remove the random effect when specifying the utility functions.

In [27]:
V_heinz41_mnl = ASC_heinz41 + B_disp *disp_heinz41 + B_feat * feat_heinz41 + B_price * price_heinz41 #+ EC_heinz41
V_heinz32_mnl = ASC_heinz32 + B_disp *disp_heinz32 + B_feat * feat_heinz32 + B_price * price_heinz32 #+ EC_heinz32
V_heinz28_mnl = ASC_heinz28 + B_disp *disp_heinz28 + B_feat * feat_heinz28 + B_price * price_heinz28 #+ EC_heinz28
V_hunts32_mnl = ASC_hunts32 + B_disp *disp_hunts32 + B_feat * feat_hunts32 + B_price * price_hunts32 #+ EC_hunts32

In [28]:
V_mnl = {0: V_heinz28_mnl,
     1: V_heinz41_mnl,
     2: V_heinz32_mnl,
     3: V_hunts32_mnl}

In [29]:
 logprob = models.loglogit (V_mnl , av , choice )
 bgm_model = bio.BIOGEME ( database_nonpanel, logprob )
results_mnl = bgm_model.estimate()

In [30]:
results_mnl.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_heinz32,0.648625,0.041539,15.614893,0.000000e+00
ASC_heinz41,-0.634556,0.092739,-6.842381,7.788881e-12
B_disp,1.364843,0.097956,13.933212,0.000000e+00
B_feat,0.939853,0.101292,9.278619,0.000000e+00
B_price,-0.519478,0.040612,-12.791345,0.000000e+00


In [31]:
tgt_mnl = {
    0: (models.logit(V_mnl, av, 0)),
    1: (models.logit(V_mnl, av, 1)),
    2: (models.logit(V_mnl, av, 2)),
    3: (models.logit(V_mnl, av, 3))
    }

sim_mnl = bio.BIOGEME(database_nonpanel, tgt_mnl)
preds_mnl = sim_mnl.simulate(theBetaValues=results_mnl.getBetaValues())
preds_mnl

,0,1,2,3
0,0.118468,0.085779,0.493973,0.301780
1,0.158921,0.115070,0.485202,0.240806
2,0.058797,0.031173,0.857035,0.052995
3,0.118468,0.085779,0.493973,0.301780
4,0.305328,0.063243,0.523913,0.107516
...,...,...,...,...
2793,0.136931,0.115868,0.463834,0.283368
2794,0.192460,0.067340,0.477352,0.262848
2795,0.050414,0.026728,0.199570,0.723288
2796,0.118707,0.062935,0.446128,0.372230


---
---

# Exercise: Capturing dynamics: Add last choice as additional variable (assume that data was observed in order), add it as fixed parameter and  estimate a mixed logit.
Basically we add a new variable and repeat the process for estimating the mixed logit.

The first step is given to us: In the following cells we are going to create a new dataset that has an additional covariate representing the alternative that was chosen before each choice situation.


In [32]:
catsup_past = catsup_pd.copy()

This functions takes a column, removes the last observation and adds a -1 at the begginning. This is how we create the lagged variable.

In [33]:
def last_choice(x):
  return pd.Series([-1]).append(x[:-1])

We apply the function `last_choice` to the dataset, but we group the dataset by the id of the household.

In [34]:
lchoice =  catsup_past.groupby('id')['choice'].apply(last_choice).reset_index()#.head(25)
catsup_past['last_choice'] = lchoice['choice']
catsup_past.head(17)

,id,disp_heinz41,disp_heinz32,disp_heinz28,disp_hunts32,feat_heinz41,feat_heinz32,feat_heinz28,feat_hunts32,price_heinz41,price_heinz32,price_heinz28,price_hunts32,choice,last_choice
0,1,0,0,0,0,0,0,0,0,4.6,3.7,5.2,3.4,0,-1
1,1,0,0,0,0,0,0,0,0,4.6,4.3,5.2,4.4,0,0
2,1,0,0,0,0,0,1,0,0,4.6,2.5,4.6,4.8,0,0
3,1,0,0,0,0,0,0,0,0,4.6,3.7,5.2,3.4,0,0
4,1,0,0,0,0,0,0,1,0,4.6,3.0,4.6,4.8,0,0
5,1,0,0,0,0,0,0,0,0,5.0,3.0,4.7,3.0,0,0
6,1,0,0,0,1,0,0,0,1,5.1,3.1,4.6,4.1,0,0
7,1,0,0,0,0,0,0,0,0,4.6,3.4,4.7,3.1,1,0
8,1,0,0,0,0,0,0,0,0,5.0,3.4,4.7,3.1,0,1
9,1,0,0,0,1,0,0,0,0,5.0,3.4,5.0,2.8,0,0


But now it is up to you how the new variable is added to the model!
Transformations? Dummy encoding? Per-alternative parameters?.